In [1]:
import json
import os
import RichContextAPI
import importlib
importlib.reload(RichContextAPI)
import re
import glob

Read in all USDA publications and partition files

In [2]:
def read_usda_pubs():
    usda_pubs_path = os.path.dirname(os.path.abspath(os.getcwd()))+'/exports/usda_pubs_list.json'
    with open(usda_pubs_path) as json_file:
        usda_pubs = json.load(json_file)
    return usda_pubs

def flatten(l):
    fl = [item for sublist in l for item in sublist]
    return fl

def read_partition_titles():
    partition_paths = glob.glob('/Users/sophierand/RCCustomers/customers/USDA/partitions/*.json')
    partition_paths
    all_partitions = []
    for p in partition_paths:
        with open(p) as json_file:
            partition_data = json.load(json_file)
            all_partitions.append(partition_data)
    all_partitions = flatten(all_partitions)
    titles = list(set([p['title'] for p in all_partitions]))
    return titles

def get_remaining_pubs():
    usda_pubs = read_usda_pubs()
    partitions = read_partition_titles()
    remaining_titles = [p for p in usda_pubs  if p['title'] not in partitions]
    return remaining_titles
    

In [3]:
def grab_md(p):
    title = p['title']
    try:
        dimensions_md = RichContextAPI.get_dimensions_md(title)
        if dimensions_md is not None:
            p.update({'dimensions':dimensions_md})
    except:
        pass
    try:
        epmc_md = RichContextAPI.get_epmc_md(title)
        if epmc_md is not None:
            p.update({'europepmc':epmc_md})
    except:
        pass
    try:
        oa_md = RichContextAPI.oa_lookup_pub_uris(title)
        if oa_md is not None:
            p.update({'openaire':oa_md})
    except:
        pass
    return p

def get_filename():
    filenames = [int(re.findall(r'\d+',g)[0]) for g in glob.glob('/Users/sophierand/RCCustomers/customers/USDA/partitions/*.json')]
    if len(filenames) == 0:
        filename = '/Users/sophierand/RCCustomers/customers/USDA/partitions/usda_pub_metadata_0.json'
    if len(filenames) > 0:
        max_filename = max(filenames)
        filename  = '/Users/sophierand/RCCustomers/customers/USDA/partitions/usda_pub_metadata_'+str(max_filename+1) + '.json'
    return filename

### MAIN

In [28]:
remaining_titles = get_remaining_pubs()
# len(remaining_titles)

In [29]:
import numpy
l = numpy.array_split(remaining_titles,len(remaining_titles)/20);

In [30]:
for sub_list in l:
    md_list = []
    for s in sub_list:
        md_entry = grab_md(s)
        md_list.append(md_entry)
    file_name = get_filename()    
    with open(file_name, 'w') as outfile:
        json.dump(md_list, outfile, indent = 2)